# Spending on Nondurables During the Great Recession

[![badge](https://img.shields.io/badge/Launch%20using%20-Econ--ARK-blue)](https://econ-ark.org/materials/nondurables-during-great-recession#launch)

<p style="text-align: center;"><small><small><small>Generator: QuARK-make/notebooks_byname</small></small></small></p>

In [1]:
# Initial imports and notebook setup, click arrow to show

import matplotlib.pyplot as plt

## Import some things from cstwMPC

from tqdm import tqdm

import numpy as np
from copy import deepcopy

import HARK # Prevents import error from Demos repo
from HARK.utilities import plot_funcs

### There Was a Big Drop in Consumption ... 
Between the second and fourth quarters of 2008, "discretionary" spending on nondurables and services in the U.S. dropped by about 4 percent -- an unprecedented collapse.  Subsequent analyses of the Great Recession concluded that it was the large drop in consumption expenditures that turned what would otherwise have been a moderate downturn into the largest economic decline since the Great Depresssion.

<!-- Following Blinder and Deaton (1987), we exclude clothing and shoes from the "nondurables" category.  By "discretionary" services, we mean those that are not mostly predetermined or imputed from other data: Specifically: recreation services, and food services and accommodations.  Data accessed on 2020-03-27 are from https://apps.bea.gov/iTable/iTable.cfm?ReqID=19&step=2#reqid=19&step=2&isuri=1&1921=underlying

Q2: (770.703+789.047+421.153+621.281+391.462)
Q4: (773.038+786.175+302.043+621.773+387.206)
(Q4-Q2)/Q2=0.0412
-->

### ... and Uncertainty Could Induce A Drop In Consumption ...  
Increased "uncertainty" has become a popular explanation of much of what happened in the Great Recession -- including this drop.  Qualitatively, it is well known that a perceived increase in labor income uncertainty should induce more saving (less consumption) for precautionary reasons.

### ... But Is the Story _Quantitatively_ Plausible?
But if explaining a 4 percent drop in discretionary consumption would require an implausibly large increase in uncertainty, the story that uncertainty explains the consumption drop is implausible.  

### Transitory Shocks, Permanent Shocks, or Unemployment
The $\texttt{ConsIndShockConsumerType}$ model incorporates three kinds of uncertainty: Unemployment spells, during which income is reduced to some small proportion of its normal level; and, for consumers who remain employed, transitory and permanent shocks with standard deviations $\sigma_{\theta}$ and $\sigma_{\psi}$.  

### The Question:
How large an increase in the standard deviation of $\sigma_{\psi}$ would be necessary to induce a 4 percent drop in consumption in one quarter?  What about $\sigma_{\theta}$?  How high would the perceived unemployment probability have to be?

The first step is to create the agents we want to solve the model for.

Model set up:
- "Standard" infinite horizon consumption/saving model, with mortality and permanent and temporary shocks to income
- Ex-ante heterogeneity in consumers' discount factors
    
With this basic setup, HARK's `IndShockConsumerType` is the appropriate subclass of $\texttt{AgentType}$. So we need to prepare the parameters to create instances of that class.


In [2]:
# Choose some calibrated parameters that roughly match steady state 
init_infinite = {
    "CRRA":1.0,                    # Coefficient of relative risk aversion 
    "Rfree":1.01/(1.0 - 1.0/240.0), # Survival probability,
    "PermGroFac":[1.000**0.25], # Permanent income growth factor (no perm growth),
    "PermGroFacAgg":1.0,
    "BoroCnstArt":0.0,
    "CubicBool":False,
    "vFuncBool":False,
    "PermShkStd":[(0.01*4/11)**0.5],  # Standard deviation of permanent shocks to income
    "PermShkCount":7,  # Number of points in permanent income shock grid
    "TranShkStd":[(0.01*4)**0.5],  # Standard deviation of transitory shocks to income,
    "TranShkCount":5,  # Number of points in transitory income shock grid
    "UnempPrb":0.07,  # Probability of unemployment while working
    "IncUnemp":0.15,  # Unemployment benefit replacement rate
    "UnempPrbRet":0.07,
    "IncUnempRet":0.15,
    "aXtraMin":0.00001,  # Minimum end-of-period assets in grid
    "aXtraMax":20,  # Maximum end-of-period assets in grid
    "aXtraCount":20,  # Number of points in assets grid,
    "aXtraExtra":[None],
    "aXtraNestFac":3,  # Number of times to 'exponentially nest' when constructing assets grid
    "LivPrb":[1.0 - 1.0/240.0],  # Survival probability
    "DiscFac":0.97,             # Default intertemporal discount factor, # dummy value, will be overwritten
    "cycles":0,
    "T_cycle":1,
    "T_retire":0,
    'T_sim':2000,  # Number of periods to simulate (idiosyncratic shocks model, perpetual youth)
    'T_age':1000,
    'IndL': 10.0/9.0,  # Labor supply per individual (constant),
    'aNrmInitMean':np.log(0.00001),
    'aNrmInitStd':0.0,
    'pLvlInitMean':0.0,
    'pLvlInitStd':0.0,
    'AgentCount':10000,
}

Now we import the class itself and make a baseline type.

In [3]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
BaselineType = IndShockConsumerType(**init_infinite)

For this exercise, we will introduce _ex ante_ heterogeneity, so the baseline type will be copied several times.

First, let's create a list with seven copies of our baseline type.

In [4]:
# A list in python can contain anything -- including consumers
num_consumer_types   = 7 # declare the number of types we want
ConsumerTypes = [] # initialize an empty list

for nn in range(num_consumer_types):
    # Now create the types, and append them to the list ConsumerTypes
    NewType = deepcopy(BaselineType)
    NewType.seed = nn # give each consumer type a different RNG seed
    ConsumerTypes.append(NewType)

Now we can give each of the consumer types their own discount factor. (This approximates the distribution of parameters estimated in ["The Distribution of Wealth and the Marginal Propensity to Consume"](https://www.econ2.jhu.edu/people/ccarroll/papers/cstwMPC)). 

In [5]:
# Seven types is enough to approximate the uniform distribution (5 is not quite enough)
from HARK.distribution import Uniform

# Calibrations from cstwMPC
bottomDiscFac  = 0.9800
topDiscFac     = 0.9934
DiscFac_list   = Uniform(bot=bottomDiscFac,top=topDiscFac).approx(N=num_consumer_types).X

# Now, assign the discount factors
for j in range(num_consumer_types):
    ConsumerTypes[j].DiscFac = DiscFac_list[j]
    ConsumerTypes[j].quiet   = True # Turn off some output

Our agents now exist and have a concept of the problem they face, but we still need them to solve that problem.

Once we have solved each type of consumer's individual problem, we need to know the distribution of wealth (and permanent income) that the population would achieve in the long run.

The cell below does both of those tasks, looping through the consumer types. For each one, it solves that type's infinite horizon model, then simulates 1000 periods to generate an approximation to the long run distribution of wealth.

In [5]:
# tqdm presents a pretty bar that interactively shows how far the calculations have gotten
for ConsumerType in tqdm(ConsumerTypes):
    ## We configured their discount factor above.  Now solve
    ConsumerType.solve(verbose=False)
    
    # Now simulate many periods to get to the stationary distribution
    ConsumerType.T_sim = 2000
    ConsumerType.initialize_sim()
    ConsumerType.simulate()

  0%|                                                                                            | 0/7 [00:03<?, ?it/s]
Internal Python error in the inspect module.
Below is the traceback from this internal error.

Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\BST\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\user\AppData\Local\Temp/ipykernel_22900/1407160588.py", line 9, in <module>
    ConsumerType.simulate()
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\HARK\core.py", line 836, in simulate
    self.sim_one_period()
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\HARK\core.py", line 541, in sim_one_period
    self.get_mortality()  # Replace some agents with "newborns"
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\HARK\core.py", line 635, in get_mortality
    self.sim_birth(who_dies)
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\HARK\ConsumptionSaving\ConsIndShockModel.py", line 1697, in sim_birth
    self.state_now['aNrm'][which_agents] = Lognormal(
  File "C:\Users\user\AppData\Roamin

TypeError: object of type 'NoneType' has no len()

With all of that setup taken care of, let's write some functions to run our counterfactual exercise and extract the information we want.

First, let's define a simple function that merely calculates the average consumption level across the entire population in the most recent simulated period.

In [ ]:
# We just merge the cNrm and pNrm lists already constructed for each ConsumerType
def calcAvgC(ConsumerTypes):
    """
    This function calculates average consumption in the economy in last simulated period,
    averaging across ConsumerTypes.
    """
    # Make arrays with all types' (normalized) consumption and permanent income level
    # The brackets indicate that the contents will be a list (in this case, of lists)
    cNrm = np.concatenate([ThisType.controls["cNrm"] for ThisType in ConsumerTypes])
    pLvl = np.concatenate([ThisType.state_now["pLvl"] for ThisType in ConsumerTypes])
    
    # Calculate and return average consumption level in the economy
    avgC = np.mean(cNrm*pLvl) # c is the ratio to p, so C = c*p
    return avgC

Now let's create a function to run the experiment we want -- change income uncertainty, and see how consumption changes.
To keep the code block below (mostly) clean, we'll describe the procedure below step by step here, with accompanying annotations in the codeblock.

1. Initialize an empty list to 
   * hold the changes in consumption that happen after parameters change, and
   * calculate average consumption before the change in uncertainty
2. Loop through the new uncertainty parameter values to assign. For each: 
   1. Assign the parameter value to the agents
   2. Re-solve the agent's model under that degree of uncertainty
   3. Construct a popn of agents distributed in the pre-crisis steady state
   4. Simulate one more period-- the first period after the change in risk.
   5. Calculate the popn avg C level given the new consumption rule
   6. Calculate the new average consumption level as percentage change vs the prior level.
   7. Return the list of percentage changes

In [ ]:
# Whenever you define a function, you should describe it (with a "docstring")
def calcConsChangeAfterUncertaintyChange(OriginalTypes,NewVals,ParamToChange):
    '''
    Calculate the change in aggregate consumption for a list of values that a 
    parameter will take on.
    
    Parameters
    ----------
    OriginalTypes : [IndShockConsumerType]
        List of consumer types, who have already had their "pre-shock" problem solved and simulated.
    NewVals : np.array
        List or array of new values that the parameter of interest will take on.
    ParamToChange : str
        Name of the income distribution parameter that will be changed.
        
    Returns
    -------
    ChangesInConsumption : [float]
        List of changes in aggregate consumption corresponding to the values in NewVals, as a percentage
        of the original aggregate consumption level.
    '''
    ChangesInConsumption = [] # Step 1
    OldAvgC = calcAvgC(OriginalTypes)

    # Step 2 (the loop over counterfactual parameter values)
    for NewVal in tqdm(NewVals):
        if ParamToChange in ["PermShkStd","TranShkStd"]:
            ThisVal = [NewVal]
        else:
            ThisVal = NewVal

        ConsumerTypesNew = deepcopy(OriginalTypes)          
        for index,ConsumerTypeNew in enumerate(ConsumerTypesNew):
            setattr(ConsumerTypeNew,ParamToChange,ThisVal) # Step 2A   
            ConsumerTypeNew.update_income_process()
            ConsumerTypeNew.solve(verbose=False) # Step 2B
            
            ConsumerTypeNew.initialize_sim() # Step 2C
            ConsumerTypeNew.aNrm = OriginalTypes[index].state_now["aNrm"]
            ConsumerTypeNew.pLvl = OriginalTypes[index].state_now["pLvl"]
            
            ConsumerTypeNew.sim_one_period() # Step 2D

        NewAvgC = calcAvgC(ConsumerTypesNew) # Step 2E
        ChangeInConsumption = 100. * (NewAvgC - OldAvgC) / OldAvgC # Step 2F
        ChangesInConsumption.append(ChangeInConsumption)

    return ChangesInConsumption # Step 3, returning the output

Our counterfactual experiment function takes three inputs-- consumer types, counterfactual values, and the name of the parameter we want to change. For the sake of convenience, let's define small functions to run the experiment for each parameter with just a single input.

In [ ]:
# Trivial functions can be useful in making the logic of your program clear
def calcConsChangeAfterPermShkChange(newVals):
    return calcConsChangeAfterUncertaintyChange(ConsumerTypes,newVals,"PermShkStd")

def calcConsChangeAfterTranShkChange(newVals):
    return calcConsChangeAfterUncertaintyChange(ConsumerTypes,newVals,"TranShkStd")

def calcConsChangeAfterUnempPrbChange(newVals):
    return calcConsChangeAfterUncertaintyChange(ConsumerTypes,newVals,"UnempPrb")

Now we can finally run our experiment.  In the cell below, we generate a plot of the change in aggregate consumption vs the (underlying) standard deviation of permanent income shocks.

In [ ]:
# Calculate the consequences of an "MIT shock" to the standard deviation of permanent shocks
ratio_min = 0.8 # minimum number to multiply uncertainty parameter by
TargetChangeInC = -4.1 # Source: see comment above
num_points = 10 # number of parameter values to plot in graphs. More=slower

# First change the variance of the permanent income shock
perm_ratio_max = 2 # Put whatever value in you want!  maximum number to multiply var of perm income shock by

perm_min = BaselineType.PermShkStd[0] * ratio_min
perm_max = BaselineType.PermShkStd[0] * perm_ratio_max

plt.ylabel('% Change in Consumption')
plt.xlabel('Std. Dev. of Perm. Income Shock (Baseline = ' + str(round(BaselineType.PermShkStd[0],2)) + ')')
plt.title('Change in Cons. Following Increase in Perm. Income Uncertainty')
plt.ylim(-50.,5.)
plt.hlines(TargetChangeInC,perm_min,perm_max)
# The expression below shows the power of python
plot_funcs([calcConsChangeAfterPermShkChange],perm_min,perm_max,N=num_points)

The figure shows that if people's beliefs about the standard deviation of permanent shocks to their incomes had changed from 0.06 (the default value) to about 0.12, the model would predict an immediate drop in consumption spending of about the magnitude seen in 2008.  

The question is whether this is a reasonable or an unreasonable magnitude for a change in uncertainty.  Some perspective on that question is offered by the large literature that attempts to estimate the magnitude of persistent or permanent shocks to household income.  The answer varies substantially across household types, countries, and time periods, but our sense of the literature is that the whole span of the territory between 0.04 and ranging nearly up to 0.20 is well populated (in the sense that substantial populations of people or countries have been estimated to experience shocks of this magnitude).

The conclusion is that, in order for an increase in permanent income uncertainty to explain the entire drop in consumption spending, uncertainty in permanent income would have to have roughly doubled between Q2 and Q4 of 2008.  While this seems rather a large increase in uncertainty, it is by no means an absurdly large increase.  And, there is no reason to rule out the possibility that people perceived a likely change in the _level_ of their permanent income as well, which of course would translate one-for-one into a change in the appropriate level of consumption.  

The point is that it is not at all implausible, as a quantitative proposition, that an increase in uncertainty could have been responsible for a substantial portion of the decline in nondurable expenditures in the Great Recesssion.  (And it is even easier for an increase in uncertainty to induce a decline in durable goods purchases.

### SOLUTION

Compute the change in transitory uncertainty and of unemployment risk that would be sufficient to make consumption fall by the appropriate amount (by making a figure similar to the one above). Brifely explain your results in intuitive terms, and discuss the implications for the measurement and interpretation of measures of "uncertainty" in consumer surveys.

In [ ]:
ratio_min = 0.8 # minimum number to multiply uncertainty parameter by
TargetChangeInC = -4.1 # Source: see comment above

tran_ratio_max = 2
unemp_ratio_max = 2

tran_min = BaselineType.TranShkStd[0] * ratio_min
tran_max = BaselineType.TranShkStd[0] * tran_ratio_max

unemp_min = BaselineType.UnempPrb * ratio_min
unemp_max = BaselineType.UnempPrb * unemp_ratio_max

In [ ]:
plt.ylabel('% Change in Consumption')
plt.xlabel('Std. Dev. of Tran. Income Shock (Baseline = ' + str(round(BaselineType.TranShkStd[0],2)) + ')')
plt.title('Change in Cons. Following Increase in Tran. Income Uncertainty')
plt.hlines(TargetChangeInC,tran_min,tran_max)

plot_funcs([calcConsChangeAfterTranShkChange],tran_min,tran_max,N=num_points)

In [ ]:
plt.ylabel('% Change in Consumption')
plt.xlabel('Std. Dev. of Unemployment Probability Shock (Baseline = ' + str(round(BaselineType.UnempPrb,2)) + ')')
plt.title('Change in Cons. Following Increase in Unemployment Probability')
plt.hlines(TargetChangeInC,unemp_min,unemp_max)

plot_funcs([calcConsChangeAfterUnempPrbChange],unemp_min,unemp_max,N=num_points)

### SOLUTION

During the Coronavirus pandemic of 2020, many governments issued "lockdown" orders to their populations to deter the spread of the disease.  A consequence was that many industries that require in-person interaction (like restaurants) had no work for their employees.  A number of governments passed "stimulus" packages that included one-time cash payments to their citizens, as a crude way to get money to people who might otherwise have faced severe financial distress.

Use the toolkit to perform an exercise designed to make a rough calculation of how such stimulus payments might affect the dynamics of aggregate consumption expenditures.  To do this, you will want to use the `ConsPrefShock` model, which allows for a temporary shock to the utility from consumption that can be used to model a one-quarter closing of restaurants and other retail businesses (your marginal utility of spending money on restaurants goes to zero if the restaurants are closed!).  

In more detail, you might incorporate a negative marginal utility shock of 10 percent in the quarter when the pandemic recession hits, followed by a full bounceback of marginal utility to its normal state one quarter later.  And, I suggest you incorporate a stimulus payment of about 5 percent of annual income for a typical consumer.  Your task is to show the path of consumer spending leading up to the pandemic quarter (which should be flat, since the pandemic is unanticipated), and then the path during the pandemic quarter and in the several quarters after it.

In [12]:
#IncShkDstn_pmf = [1]
#IncShkDstn_X = [[0],[0.05]]
#IncShkDstn_seed = 37

# 5 quarters before pandemic quarter, 1 quarter of marginal utility shock, 
# 5 quarters after pandemic quarter back to normal state

#Baseline2Type.IncShkDstn = [IncShkDstn_pmf,IncShkDstn_X,IncShkDstn_seed]


from HARK.ConsumptionSaving.ConsPrefShockModel import IndShockConsumerType
Baseline2Type = IndShockConsumerType(**init_infinite)
Baseline2Type.PrefShkDstn = [[0,0,0,0,0,1,1,0,0,0,0],[0,0,0,0,0,-0.1,0,0,0,0,0]]

Baseline2Type.solve(verbose=False)
    
Baseline2Type.initialize_sim()
Baseline2Type.simulate()
Baseline2Type

GPFRaw                 = 0.983799 
GPFNrm                 = 0.987147 
GPFAggLivPrb           = 0.979700 
Thorn = APF            = 0.983799 
PermGroFacAdj          = 0.996609 
uInvEpShkuInv          = 1.000000 
VAF                    = 0.965958 
WRPF                   = 0.067617 
DiscFacGPFNrmMax       = 0.982630 
DiscFacGPFAggLivPrbMax = 0.990099 


<HARK.ConsumptionSaving.ConsIndShockModel.IndShockConsumerType object at 0x178d48c6b20.
Parameters:
cycles: 0
CRRA: 1.0
Rfree: 1.014225941422594
DiscFac: 0.97
LivPrb: [0.9958333333333333]
PermGroFac: [1.0]
BoroCnstArt: 0.0
MaxKinks: 400
AgentCount: 10000
aNrmInitMean: -11.512925464970229
aNrmInitStd: 0.0
pLvlInitMean: 0.0
pLvlInitStd: 0.0
PermGroFacAgg: 1.0
T_age: 1000
T_cycle: 1
PerfMITShk: False
aXtraMin: 1e-05
aXtraMax: 20
aXtraNestFac: 3
aXtraCount: 20
aXtraExtra: [None]
PermShkStd: [0.06030226891555272]
PermShkCount: 7
TranShkStd: [0.2]
TranShkCount: 5
UnempPrb: 0.07
UnempPrbRet: 0.07
IncUnemp: 0.15
IncUnempRet: 0.15
tax_rate: 0.0
T_retire: 0
vFuncBool: False
CubicBool: False
neutral_measure: False
T_sim: 2000
IndL: 1.1111111111111112>

In [13]:
cNrm = [Baseline2Type.controls["cNrm"]]
# pLvl = [Baseline2Type.controls["pLvl"]]
cTotal = np.mean(cNrm)
#cTotal = np.mean(cNrm*pLvl)

In [14]:
# Calculate and return average consumption level in the economy
avgC = np.mean(cNrm*pLvl) # c is the ratio to p, so C = c*p
quarters = np.linspace(-5,6,12)

for index,Problem2Type in enumerate(Problem2Types):
    Problem2Type.aNrm = Problem2Type[index].state_now["aNrm"]
    Problem2Type.pLvl = Problem2Type[index].state_now["pLvl"]

    ConsumerTypeNew.sim_one_period() # Step 2D

plt.figsize(8,5)
plt.plot(quarters,???,'--k')
plt.xlabel('Time')
plt.ylabel('Aggregate Coinsumption')
plt.show

SyntaxError: invalid syntax (Temp/ipykernel_22900/563653773.py, line 12)